# 순환 신경망

In [ ]:
from tensorflow.keras.datasets import imdb

# imdb data 불러오기

(train_input, train_target), (test_input, test_target) = imdb.load_data(num_words = 500)

print(train_input.shape, test_input.shape)

print(train_input[0])

print(train_target[:20])

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split


train_input, val_input, train_target, val_target = train_test_split(train_input,train_target, test_size=0.2, random_state = 42)

lengths = np.array([len(x) for x in train_input])
print(np.mean(lengths), np.median(lengths))

plt.hist(lengths)
plt.xlabel('length')
plt.ylabel('frequency')

In [ ]:
# 시퀀스 패딩

from tensorflow.keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_input,maxlen=100)

print(train_seq.shape)

print(train_seq[0]) 

print(train_input[0][-10:]) # 원본의 끝 부분.

# 위의 결과를 보면, pad 처리를 하면 문장의 앞 부분을 삭제하는 것을 알 수 있음.

print(train_seq[5])

# 위의 결과를 보면, 데이터가 부족하면 앞 부분을 0으로 채움.
# pad 클래스는 뒷부분의 데이터를 더 중요하게 보고 있다는 것을 알 수 있음.

val_seq = pad_sequences(val_input, maxlen=100)



In [ ]:

from tensorflow import keras

model = keras.Sequential()

model.add(keras.layers.SimpleRNN(8,input_shape=(100,500)))
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
# 원핫 인코딩

train_onehot = keras.utils.to_categorical(train_seq)

print(train_onehot.shape)

print(train_onehot[0][0][:12])
print(np.sum(train_onehot[0][0])) # 단한개만 1로 변경됨을 알 수 있음.

val_onehot = keras.utils.to_categorical(val_seq)


model.summary()

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer=rmsprop, loss='binary_crossentropy',metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-simplernn-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)

history = model.fit(train_onehot,train_target,epochs=100,batch_size=64,validation_data=(val_onehot,val_target),callbacks=[checkpoint_cb, early_stopping_cb])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
# 임베딩

model2 = keras.Sequential()

model2.add(keras.layers.Embedding(500,16,input_length=100)) # 토큰(단어)를 실수 벡터로 변환하여 처리하는 방식 (위의 원핫보다 데이터 사이즈가 작아서 유리하다. 사이즈가 작은데도 정밀도는 높은거 같다.)
model2.add(keras.layers.SimpleRNN(8))
model2.add(keras.layers.Dense(1,activation='sigmoid'))

model2.summary()



In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model2.compile(optimizer=rmsprop, loss='binary_crossentropy',
               metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-embedding-model.h5',
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,
                                                  restore_best_weights=True)

history = model2.fit(train_seq, train_target, epochs=100, batch_size=64,
                     validation_data=(val_seq, val_target),
                     callbacks=[checkpoint_cb, early_stopping_cb])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()